# Power law in food metabolome


## Part 1 : Code for fitting power law distribution between Intensity and m/z


In [ ]:
# All the necessary python libraries
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.formula.api import ols
import math
from __future__ import print_function

import numpy as np
import scipy.stats


In [ ]:
# Reading a .xlsx file
data= pd.read_excel(".xlsx")
# Reading a .ods file
data= pd.read_excel(".ods",engine="odf")

In [ ]:
# Choosing only the columns needed to run the algorithm
data=data[["m/z","I"]]# Cocoa and Thearubigins FT-ICR dataset
data=data[["Max. m/z","Area"]]# Cocoa LCMS
data=data[["m/z meas.","I"]]# Wine FTICR

In [ ]:
# For LCMS Cocoa
data["RT [min]"]=data["RT [min]"].astype("float")

In [ ]:
# Filtering out the rows with Intensity 0 for wine dataset
data=data[~(data["I"]==0)]

In [ ]:
data

In [ ]:
fig,ax1=plt.subplots()
sns.histplot(data["m/z meas."])
ax1.set_ylabel("frequency")


In [ ]:
fig,ax3=plt.subplots()
sns.histplot(data["RT [min]"])
ax3.set_ylabel("frequency")

In [ ]:
data.describe()

In [ ]:
fig,ax2=plt.subplots()
sns.histplot(data["I"],log_scale=True)
ax2.set_xlabel("Intensity")
ax2.set_ylabel("Frequency")

In [ ]:
data.shape

In [ ]:
data=data.sort_values(by="m/z",ascending=True)


In [ ]:
# Plotting Intensity vs m/z
fig,axe=plt.subplots(figsize=(7,5))
axe.scatter(data['m/z'],data['I'],s=50,color="red")
axe.set_title("    Intensity as a function of m/z")
axe.set_xlabel("m/z")
axe.set_ylabel("Intensity")
#plt.suptitle("Cocoa FTICR 24h")



In [ ]:
#modifying the x and y axis into a logarithmic scale
log_mz=np.log(data["m/z"])
log_intensity=np.log(data["I"])
fig,a=plt.subplots(figsize=(7,5))
a.scatter(log_mz,log_intensity,s=50,color="red")
a.set_title("$log(Intensity)=f(log(m/z))$")
a.set_xlabel(" log_m/z")
a.set_ylabel("log_Intensity")


### $$y=a * x^b $$
$$frequency = a * Intensity ^b$$
$$log(frequency) =log( a * Intensity ^b)$$
$$log(frequency) = log(a) + log(Intensity)^b$$
$$log(frequency) = log(a) + b* log(Intensity)$$
                                     
                                      
                                            

In [ ]:
# Getting the log of the data to fit a staright line
data_log = np.log(data[["I","m/z"]])
data_log.rename(columns={"m/z":"coefficient"},inplace=True)

In [ ]:
# Fitting the OLS model and getting the parameters
model = ols('I ~ coefficient',data=data_log).fit()
model.summary()


In [ ]:
# Rsquared vale
rsquared=model.rsquared


In [ ]:
# putting the parameters slope and intercept into a dictionary
params=dict(model.params)


In [ ]:


params

$$log(Intensity) = 16.0896 + 0.0590	* log(m/z)$$
$$e^{log(Intensity)} = e^{16.0896 + 0.0590	* log(m/z)}$$
$$ Intensity= e^{16.0896 + 0.0590	* log(m/z)}$$
$$ Intensity= e^{16.0896} +e^{ 0.0590	* log(m/z)}$$
$$ Intensity= 9719065.2099 * m/z^{0.0590}$$

In [ ]:
#Linearising the intercept
exp=math.exp(params['Intercept'])

In [ ]:
# Getting the predicted intensities by fitting the model parameters into the linear power law equation
I_predicted = exp * data[['m/z']]**(params['coefficient'])
data.insert(loc=2,value=I_predicted,column="I_predicted")

In [ ]:
# Intensity vs Predicted intensity plot
plt.rcParams.update({"font.size":18,"font.family":"serif"})
fig,ax=plt.subplots(figsize=(10,6))

ax.scatter((data[["m/z"]]),(data[["I"]]),s=50,color="red",label="Intensity")
ax.scatter((data[["m/z"]]),(data[["I_predicted"]]),lw=3,label="Intensity Predicted",color="blue")
#ax.plot(data[['m/z']],data[["I_predicted"]],r'g*',markersize=10)
ax.legend(loc=1)
#ax.grid(True)
ax.set_title("                  Actual Intensity vs Predicted  ")
ax.set_ylabel("Intensity")
ax.set_xlabel("m/z")

In [ ]:
# Intensity vs Predicted Intensity in log scale 
plt.rcParams.update({"font.size":18,"font.family":"serif"})
fig,ax=plt.subplots(figsize=(10,6))

ax.scatter(np.log(data[["m/z meas."]]),np.log(data[["I"]]),s=50,color="red",label="Intensity")
ax.scatter(np.log(data[["m/z meas."]]),np.log(data[["I_predicted"]]),lw=3,label="Intensity Predicted",color="blue")
#ax.plot(data[['m/z']],data[["I_predicted"]],r'g*',markersize=10)
ax.legend(loc=1)

ax.set_title("Actual Intensity vs Predicted ")
ax.set_ylabel("log(Intensity)")
ax.set_xlabel("log(m/z)")

In [ ]:
# predicted intensity plots 
plt.plot(data["m/z"],data['I_predicted'],"r.")
plt.xlabel("m/z")
plt.ylabel("Predicted intensity")

In [ ]:
# actual intensity plot
fig,ax1=plt.subplots(figsize=(4,4))
plt.xlabel("m/z")
plt.ylabel("Intensity")
plt.plot(data["m/z"],data['I'],"b.")

In [ ]:
fig,ax1=plt.subplots(figsize=(4,4))
plt.xlabel("m/z")
plt.ylabel("Intensity")
plt.plot(data["m/z"],data['I_predicted'],"r.")

In [ ]:
# Actual intensity in log plot
fig,ax2=plt.subplots(figsize=(4,4))
plt.plot(np.log(data["m/z"]),np.log(data['I']),"b.")
plt.xlabel("log (m/z)")
plt.ylabel("log(Intensity)")
#ax2.ticklabel_format(style="plain")
#plt.xticks(rotation=45)
#axe.xticks(rotation=45)
plt.show()

In [ ]:
# Predicted intensity in log plot
fig,ax2=plt.subplots(figsize=(4,4))
plt.plot(np.log(data["m/z"]),np.log(data['I_predicted']),"r.")
plt.xlabel("log (m/z)")
plt.ylabel("log( Predicted Intensity)")
#ax2.ticklabel_format(style="plain")
#plt.xticks(rotation=45)
#axe.xticks(rotation=45)
plt.show()


## Part 2: Filtering tool for fitting power law between Intensity and Intensity frequency

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from statsmodels.formula.api import ols
from pathlib import Path
import math
import os
import sys
import warnings



In [ ]:
file_path="C://Users//Athira Shankar//Downloads//more_data//pos1981.xlsx"
sheet_name="Sheet1"
img_path="C:/Users/Athira Shankar/Downloads/"


In [ ]:
data= pd.read_excel(file_path,sheet_name=sheet_name)
data=data[~(data["I"]==0)]
# run for Cocoa LCMS only
#data.rename(columns={"I":"I_"},inplace=True)
#data.rename(columns={"Area":"I"},inplace=True)
input_data=data.sort_values(by=["I"],ascending=True)
interval=input("Please enter the interval width")
n=input("Please enter the number of filters ")

In [ ]:
x_coeff=[]
intercept=[]
exp_intercept=[]
rsquared=[]
i=1
while(i<=n):
    num_class =int(interval)
    range_interval =np.max(input_data["I"]) - np.min(input_data["I"])
    if range_interval % num_class ==0: 
         num_class =num_class+1
    width =math.ceil(range_interval /num_class)
    low_class =np.arange(np.min(input_data["I"]),np.max(input_data["I"]),width)
    upper_class = low_class + width 
    intensity=np.array(input_data["I"])
    class_pairs =list(zip(low_class,upper_class))
    list_result =[]
    for (low_bound, upper_bound) in class_pairs:
        result =np.count_nonzero(intensity[(intensity>=low_bound) & (intensity<=upper_bound)])
        list_result.append(result)
    intervals=pd.DataFrame()
    intervals["Intervals"]=class_pairs
    intervals["low_class intervals"]=low_class
    intervals["upper_class intervals"]=upper_class
    intervals["frequency"]=list_result
    intervals["Intervals"]=intervals["Intervals"].astype("str").str.replace("(","[").str.replace(")","]")
    mean=np.mean(intervals["frequency"])
    intervals_x=intervals[intervals["frequency"]>=mean]
    intensity_filtered=input_data[(input_data["I"] <=intervals_x['upper_class intervals'].iloc[len(intervals_x)-1] )]
    data_log=pd.DataFrame()
    data_log = np.log(intervals[["frequency","low_class intervals"]])
    data_log.rename(columns={"low_class intervals":"low_class_intervals"},inplace=True)
    np.seterr(divide = 'ignore')
    data_log["frequency"][np.isneginf(data_log["frequency"])]=0
    
    model = ols('frequency ~ low_class_intervals',data=data_log).fit()
    params=dict(model.params)
    rsquared.append(model.rsquared)
    x_coeff.append(params["low_class_intervals"])
    intercept.append(params["Intercept"])
    exp=math.exp(params["Intercept"])
    exp_intercept.append(exp)
    frequency_predicted = exp * input_data[['I']]**(params["low_class_intervals"])
    intervals.insert(loc=2,value=frequency_predicted,column="frequency_predicted")
    intervals["frequency_predicted"]=np.round(intervals["frequency_predicted"])
    
    input_data=intensity_filtered
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))
    fig.tight_layout(h_pad=2,w_pad=5)
    plt.subplots_adjust(top=0.90)

    fig.suptitle("Intensity vs frequency plots")


    ax1.scatter((intervals[["low_class intervals"]]),(intervals[["frequency"]]),color="red",label="Intensity")
    ax1.plot((intervals[["low_class intervals"]]),(intervals[["frequency"]]),color="red",label="Intensity")
    #axs[0, 0].set_title('Frequency vs Intensity')
    ax1.set_xlabel('Intensity Filter level '+ str(i))
    ax1.set_ylabel('Frequency Filter level '+ str(i))
    ax2.scatter(np.log(intervals[["low_class intervals"]]),np.log(intervals[["frequency"]]),color="b")
    ax2.plot(np.log(intervals[["low_class intervals"]]),np.log(intervals[["frequency"]]),color="b")
    ax2.set_xlabel('Intensity (log) Filter level '+str(i))
    ax2.set_ylabel(' Frequency(log) Filter level '+str(i))
    plt.show()
    
    i=i+1


    
    


In [ ]:
Filter_level=["Filter level 1","Filter level 2","Filter level 3","Filter level 4","Filter level 5"]
fig2 = plt.figure()
plt.plot(Filter_level,exp_intercept)
plt.suptitle("Intercept")
fig2 = plt.figure()
plt.plot(Filter_level,x_coeff)
plt.suptitle("Intensity coefficient")
fig2 = plt.figure()
plt.plot(Filter_level,rsquared)

plt.ylabel("Rsquared")
plt.suptitle("Rsquared plot Wine 1981 positive 30 intervals")
#fig2.savefig(image_path+"Rsquared.png")

In [ ]:
print("Operation completed and the plots have been saved")